In [1]:
!pip install datasets transformers seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 5.1 MB/s 
     |████████████████████████████████| 5.3 MB 60.2 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 115 kB 66.0 MB/s 
     |████████████████████████████████| 163 kB 65.2 MB/s 
     |████████████████████████████████| 212 kB 72.0 MB/s 
     |████████████████████████████████| 127 kB 69.0 MB/s 
     |████████████████████████████████| 7.6 MB 77.4 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=c0d897dbdd4dd29c3a2c76924d5778df52d4f5d4b75da46e8c67fe974561ce8e
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.

Then you need to install Git-LFS. Uncomment the following instructions:

In [2]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.


In [3]:
!huggingface-cli login



        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your termin

# **Fine-tuning a model on a token classification task**

In [4]:
import transformers

print(transformers.__version__)

4.23.0


In [59]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "amichailidis/greek_legal_bert_v2-finetuned-ner"
batch_size = 16

## **Loading the dataset**

In [6]:
#connect to G-Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
from pathlib import Path
import re

def read_wnut(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\s?\n', raw_text)

    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            #print(line.split(' '))
            token, tag= line.split(' ')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

texts, tags = read_wnut('/content/gdrive/MyDrive/Colab Notebooks/NER/ORG_IOB_tags_ER_data.txt')

In [13]:
texts = [ele for ele in texts if ele != ['']]
tags = [ele for ele in tags if ele != ['']]

In [14]:
print(len(texts))
print(len(tags))

414
414


Have a look on data! 

In [15]:
print(texts[2][0:100], tags[2][0:100], sep='\n')

['"', 'Το', 'Υπουργείο', 'Εργασίας', ',', 'Κοινωνικής', 'Ασφάλισης', 'και', 'Κοινωνικής', 'Αλληλεγγύης', 'διαρθρώνεται', ',', 'ως', 'εξής', ':', '2.', 'β', '.', 'Γενική', 'Γραμματεία', 'Κοινωνικής', 'Αλληλεγγύης', 'Στη', 'Γενική', 'Γραμματεία', 'Κοινωνικής', 'Αλληλεγγύης', 'υφίσταται', 'θέση', 'Τομεακού', 'Γραμματέα', '.', 'Ο', 'Τομεακός', 'Γραμματέας', 'είναι', 'επικεφαλής', 'της', 'διοικητικής', 'ιεραρχίας', 'της', 'Γενικής', 'Γραμματείας', 'Κοινωνικής', 'Αλληλεγγύης', 'και', 'είναι', 'υπεύθυνος', 'για', 'την', 'εκτέλεση', 'και', 'τη', 'διοικητική', 'εφαρμογή', 'της', 'πολιτικής', 'των', 'υπηρεσιών', 'των', 'οποίων', 'προΐσταται', ',', 'όπως', 'αυτή', 'καθορίζεται', 'από', 'την', 'Κυβέρνηση', 'και', 'τα', 'όργανά', 'της', '.', 'Η', 'Γενική', 'Γραμματεία', 'Κοινωνικής', 'Αλληλεγγύης', 'αποτελείται', 'από', 'τις', 'ακόλουθες', 'υπηρεσίες', ':', 'αα', ')', 'Γραφείο', 'Τομεακού', 'Γραμματέα', ',', 'το', 'οποίο', 'στελεχώνεται', 'από', 'υπαλλήλους', 'του', 'Υπουργείου', 'και', 'επικουρεί'

In [16]:
for i in range(len(tags)-1,-1,-1):
  if 'B-ORG' not in tags[i]:
    texts.pop(i)
    tags.pop(i)
  else:
    continue

In [17]:
for i in range(len(tags)):
  if len(texts[i]) == len(tags[i]):
    continue
  else:
    print(i)

In [18]:
print(len(texts))
print(len(tags))

413
413


In [19]:
print(texts[2][0:100], tags[2][0:100], sep='\n')

['"', 'Το', 'Υπουργείο', 'Εργασίας', ',', 'Κοινωνικής', 'Ασφάλισης', 'και', 'Κοινωνικής', 'Αλληλεγγύης', 'διαρθρώνεται', ',', 'ως', 'εξής', ':', '2.', 'β', '.', 'Γενική', 'Γραμματεία', 'Κοινωνικής', 'Αλληλεγγύης', 'Στη', 'Γενική', 'Γραμματεία', 'Κοινωνικής', 'Αλληλεγγύης', 'υφίσταται', 'θέση', 'Τομεακού', 'Γραμματέα', '.', 'Ο', 'Τομεακός', 'Γραμματέας', 'είναι', 'επικεφαλής', 'της', 'διοικητικής', 'ιεραρχίας', 'της', 'Γενικής', 'Γραμματείας', 'Κοινωνικής', 'Αλληλεγγύης', 'και', 'είναι', 'υπεύθυνος', 'για', 'την', 'εκτέλεση', 'και', 'τη', 'διοικητική', 'εφαρμογή', 'της', 'πολιτικής', 'των', 'υπηρεσιών', 'των', 'οποίων', 'προΐσταται', ',', 'όπως', 'αυτή', 'καθορίζεται', 'από', 'την', 'Κυβέρνηση', 'και', 'τα', 'όργανά', 'της', '.', 'Η', 'Γενική', 'Γραμματεία', 'Κοινωνικής', 'Αλληλεγγύης', 'αποτελείται', 'από', 'τις', 'ακόλουθες', 'υπηρεσίες', ':', 'αα', ')', 'Γραφείο', 'Τομεακού', 'Γραμματέα', ',', 'το', 'οποίο', 'στελεχώνεται', 'από', 'υπαλλήλους', 'του', 'Υπουργείου', 'και', 'επικουρεί'

In [20]:
for i in range(len(tags)-1,-1,-1):
  if tags[i].count('B-ORG') == 1 and len(tags[i]) > 50:
    texts.pop(i)
    tags.pop(i)
  else:
    continue

In [21]:
for i in range(len(tags)):
  if len(texts[i]) == len(tags[i]):
    continue
  else:
    print(i)

In [22]:
print(len(texts))
print(len(tags))

410
410


## **Preprocessing the data**

Split Dataset

In [23]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

Dictionary of Tags to IDs and vise-versa

In [24]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [60]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/523 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/509k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--amichailidis--greek_legal_bert_v2-finetuned-ner/snapshots/1a4b37fd950ed7fba4427c2f11a597195cd471c5/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--amichailidis--greek_legal_bert_v2-finetuned-ner/snapshots/1a4b37fd950ed7fba4427c2f11a597195cd471c5/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--amichailidis--greek_legal_bert_v2-finetuned-ner/snapshots/1a4b37fd950ed7fba4427c2f11a597195cd471c5/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--amichailidis--greek_legal_bert_v2-finetuned-ner/snapshots/1a4b37fd950ed7fba4427c2f11a597195cd471c5/tokenizer_config.json


Since our inputs have already been split into words, we pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [61]:
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Encode our Dataset

In [62]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [63]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)

In [64]:
print(train_dataset[1]['input_ids'],train_dataset[1]['labels'])

tensor([    2,     6,   432,  1434,  1528, 18992,   668,  4205,    30,    21,
           18,   122,    18,  3233,  3249,  1796,    18,   525,  3233,  3249,
         1796,  6679,   966,  2588,  1429,   999,  1296,   458,  7424,   779,
         1784,    18,   135,  6930,  1429,  1919,  2074,   522, 11364,   426,
         2117, 16384,   426,  3521,  2817,  1796,   414,   522,  5270,   453,
          434,  2029,   414,   421,  4528,  1365,   426,  1730,   451,  1110,
          451,  2680,  6936,    16,   676,   686,  3365,   422,   434,  1172,
          414,   472,  2110,   426,    18,   127,  3233,  3249,  1796,  4445,
          422,   495,  6039,  1431,    30,   683,    18,  1711,  2588,  1429,
          999,  1296,    16,   432,   843, 26734,   422,  3071,   458,  2632,
          744,   787,  2136,  3662,   591,   414,  3952,   737,   547,  1097,
         1429,   612,  1296,   525,  3244,   451,  3974,   412,    18,  9360,
           18,  2055,  1796,    16,   127,   688,  7424,   779, 

In [30]:
len(train_dataset[1]['input_ids'])


321

## **Fine-tuning the model**

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [65]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(unique_tags), id2label= id2tag, label2id= tag2id)

Downloading:   0%|          | 0.00/808 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--amichailidis--greek_legal_bert_v2-finetuned-ner/snapshots/1a4b37fd950ed7fba4427c2f11a597195cd471c5/config.json
Model config BertConfig {
  "_name_or_path": "amichailidis/greek_legal_bert_v2-finetuned-ner",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-ORG",
    "1": "O",
    "2": "I-ORG"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-ORG": 0,
    "I-ORG": 2,
    "O": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.23.0",
  "type_vocab_size

Downloading:   0%|          | 0.00/449M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--amichailidis--greek_legal_bert_v2-finetuned-ner/snapshots/1a4b37fd950ed7fba4427c2f11a597195cd471c5/pytorch_model.bin
All model checkpoint weights were used when initializing BertForTokenClassification.

All the weights of BertForTokenClassification were initialized from the model checkpoint at amichailidis/greek_legal_bert_v2-finetuned-ner.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForTokenClassification for predictions without further training.


To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [66]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"greek_legal_bert_v2-finetuned-ner-V4",
    evaluation_strategy = "steps", 
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_steps=25,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=True,
    metric_for_best_model='eval_f1',
    load_best_model_at_end = True,)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [67]:
from datasets import load_metric
metric = load_metric("seqeval")

This metric takes list of labels for the predictions and references:

In [68]:
labels = [id2tag[i] for i in train_labels[0] if i!= -100]
metric.compute(predictions=[labels], references=[labels])

{'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 4},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [69]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [70]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,           # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/huggingface_hub/repository.py:708: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.11.
  FutureWarning,
Cloning https://huggingface.co/amichailidis/greek_legal_bert_v2-finetuned-ner-V4 into local empty directory.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 328
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 210


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
25,No log,0.124631,0.826401,0.818996,0.822682,0.968003
50,No log,0.072902,0.871622,0.924731,0.897391,0.980932
75,No log,0.055345,0.897785,0.944444,0.920524,0.985112
100,No log,0.059109,0.898973,0.940860,0.919440,0.985242
125,No log,0.059840,0.901724,0.937276,0.919156,0.984850
150,No log,0.062843,0.906412,0.937276,0.921586,0.984197
175,No log,0.063564,0.903114,0.935484,0.919014,0.984328
200,No log,0.065888,0.903282,0.937276,0.919965,0.984459


***** Running Evaluation *****
  Num examples = 82
  Batch size = 16
***** Running Evaluation *****
  Num examples = 82
  Batch size = 16
***** Running Evaluation *****
  Num examples = 82
  Batch size = 16
***** Running Evaluation *****
  Num examples = 82
  Batch size = 16
***** Running Evaluation *****
  Num examples = 82
  Batch size = 16
***** Running Evaluation *****
  Num examples = 82
  Batch size = 16
***** Running Evaluation *****
  Num examples = 82
  Batch size = 16
***** Running Evaluation *****
  Num examples = 82
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=210, training_loss=0.2290572030203683, metrics={'train_runtime': 221.9784, 'train_samples_per_second': 14.776, 'train_steps_per_second': 0.946, 'total_flos': 537337141644960.0, 'train_loss': 0.2290572030203683, 'epoch': 10.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [71]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 82
  Batch size = 16


{'eval_loss': 0.06588701158761978,
 'eval_precision': 0.9032815198618307,
 'eval_recall': 0.9372759856630825,
 'eval_f1': 0.9199648197009676,
 'eval_accuracy': 0.9844586652736058,
 'eval_runtime': 1.7909,
 'eval_samples_per_second': 45.787,
 'eval_steps_per_second': 3.35,
 'epoch': 10.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [72]:
predictions, labels, _ = trainer.predict(val_dataset)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

***** Running Prediction *****
  Num examples = 82
  Batch size = 16


{'ORG': {'precision': 0.9032815198618307,
  'recall': 0.9372759856630825,
  'f1': 0.9199648197009676,
  'number': 558},
 'overall_precision': 0.9032815198618307,
 'overall_recall': 0.9372759856630825,
 'overall_f1': 0.9199648197009676,
 'overall_accuracy': 0.9844586652736058}

## Push to Hub Directly


In [74]:
trainer.push_to_hub()

Saving model checkpoint to greek_legal_bert_v2-finetuned-ner-V4
Configuration saved in greek_legal_bert_v2-finetuned-ner-V4/config.json
Model weights saved in greek_legal_bert_v2-finetuned-ner-V4/pytorch_model.bin
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.9032815198618307}, {'name': 'Recall', 'type': 'recall', 'value': 0.9372759856630825}, {'name': 'F1', 'type': 'f1', 'value': 0.9199648197009676}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9844586652736058}]}
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/429M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/amichailidis/greek_legal_bert_v2-finetuned-ner-V4
   7b52bfc..a0a0754  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/amichailidis/greek_legal_bert_v2-finetuned-ner-V4
   7b52bfc..a0a0754  main -> main



In [75]:
tokenizer.push_to_hub('greek_legal_bert_v2-finetuned-ner-V4')

tokenizer config file saved in greek_legal_bert_v2-finetuned-ner-V4/tokenizer_config.json
Special tokens file saved in greek_legal_bert_v2-finetuned-ner-V4/special_tokens_map.json
Uploading the following files to amichailidis/greek_legal_bert_v2-finetuned-ner-V4: tokenizer_config.json,tokenizer.json,vocab.txt,special_tokens_map.json


CommitInfo(commit_url='https://huggingface.co/amichailidis/greek_legal_bert_v2-finetuned-ner-V4/commit/ae73c4cc631356aacf0601c9cc11cf578599a940', commit_message='Upload tokenizer', commit_description='', oid='ae73c4cc631356aacf0601c9cc11cf578599a940', pr_url=None, pr_revision=None, pr_num=None)

## Save Model Locally

In [ ]:
model.save_pretrained("gdrive/MyDrive/path/to/model2")


Configuration saved in gdrive/MyDrive/path/to/model2/config.json
Model weights saved in gdrive/MyDrive/path/to/model2/pytorch_model.bin


In [ ]:
tokenizer.save_pretrained("gdrive/MyDrive/path/to/model2")


tokenizer config file saved in gdrive/MyDrive/path/to/model2/tokenizer_config.json
Special tokens file saved in gdrive/MyDrive/path/to/model2/special_tokens_map.json


('gdrive/MyDrive/path/to/model2/tokenizer_config.json',
 'gdrive/MyDrive/path/to/model2/special_tokens_map.json',
 'gdrive/MyDrive/path/to/model2/vocab.txt',
 'gdrive/MyDrive/path/to/model2/added_tokens.json',
 'gdrive/MyDrive/path/to/model2/tokenizer.json')

## Perform tests

In [ ]:
model_test = AutoModelForTokenClassification.from_pretrained('/content/gdrive/MyDrive/path/to/model2')    
tokenizer_test = AutoTokenizer.from_pretrained('/content/gdrive/MyDrive/path/to/model2')

loading configuration file /content/gdrive/MyDrive/path/to/model2/config.json
Model config BertConfig {
  "_name_or_path": "/content/gdrive/MyDrive/path/to/model2",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "I-ORG",
    "2": "B-ORG"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-ORG": 2,
    "I-ORG": 1,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.22.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 35000
}

loading weights file /content/gdrive/MyDrive/path/to/model2/pytorch_model.bin
A

In [ ]:
from transformers import TokenClassificationPipeline

pipe = TokenClassificationPipeline(model=model_test, tokenizer=tokenizer_test)
pipe("(3) Γραφείο Συμβούλου Ακεραιότητας (ΓΣΑ)")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-ORG',
  'score': 0.99988246,
  'index': 4,
  'word': 'γραφειο',
  'start': 4,
  'end': 11},
 {'entity': 'I-ORG',
  'score': 0.9998987,
  'index': 5,
  'word': 'συμβουλου',
  'start': 12,
  'end': 21},
 {'entity': 'I-ORG',
  'score': 0.9999453,
  'index': 6,
  'word': 'ακεραιοτητας',
  'start': 22,
  'end': 34},
 {'entity': 'B-ORG',
  'score': 0.999566,
  'index': 8,
  'word': 'γ',
  'start': 36,
  'end': 37},
 {'entity': 'I-ORG',
  'score': 0.9928483,
  'index': 9,
  'word': '##σα',
  'start': 37,
  'end': 39}]